In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import torch
import evaluate
import numpy as np


In [3]:
from transformers import AutoModelForQuestionAnswering
import torch


model_name = "meta-llama/Llama-3.2-1B"
# Access_token goes here


tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    use_auth_token=access_token
)


for param in model.parameters():
    param.requires_grad = False


for param in model.qa_outputs.parameters():
    param.requires_grad = True
    

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

#Took the suggestion of training only last layer by freezing other parameters to overcome "Out of Memory issue" from fellow students.


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:796: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForQuestionAnswering were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['embed_tokens.weight', 'layers.0.input_layernorm.weight', 'layers.0.mlp.down_proj.weight', 'layers.0.mlp.gate_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.0.post_attention_layernorm.weight', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.o_proj.weight', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.v_proj.weight', 'layers.1.input_layernorm.weight', 'layers.1.mlp.down_proj.weight', 'layers.1.mlp.gate_proj.weight', 'layers.1.mlp.up_proj.weight', 'layers.1.post_attention_layernorm.weight', 'layers.1.self_attn.k_proj.weight', 'layers.1.self_attn.o_proj.weight', 'layers.1.self_attn.q_proj.weight', 'layers.1.self_attn.v_proj.weight', 'layers.10.input_layernorm.weight', 'layers.10.mlp.down_proj.weight', 'layers.10.mlp.gate_proj.weight', 'layers.10.mlp.up_proj.weight', 'layers.10.post_attention_layernorm.weight', 'layers.10.s

Total Parameters: 1,235,818,498
Trainable Parameters: 4,098


In [4]:
# model = model.full()
# model.gradient_checkpointing_enable()
model.config.use_cache = False


In [5]:
print(torch.cuda.memory_allocated()) 
print(torch.cuda.memory_reserved()) 

0
0


In [6]:
import random
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    Trainer,
    TrainingArguments,
    default_data_collator,
)
import evaluate

In [7]:
# from datasets import load_dataset

squad_v2 = load_dataset("rajpurkar/squad_v2")

# print(squad_v2)

SEED = 1
split_squad_v2 = squad_v2["train"].train_test_split(test_size=0.2, seed=SEED)
train_squad_v2 = split_squad_v2["train"]
test_squad_v2 = split_squad_v2["test"]


import copy

testing_copy_2 = copy.deepcopy(split_squad_v2["test"])
testing_copy_2 = testing_copy_2.select(range(30))
# print(testing_copy_2)
print(train_squad_v2.column_names)

print(testing_copy_2["answers"][0]['text'][0])


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

['id', 'title', 'context', 'question', 'answers']
Imperial's Royal Charter


In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# def preprocess_batch_llama(batch):
#     
#     inputs = [
#         f"Context: {context}\nQuestion: {question}\nAnswer: {answer.get('text')}"
#         for context, question, answer in zip(batch['context'], batch['question'], batch['answers'])
#     ]
    
#     
#     tokenized = tokenizer(
#         inputs,
#         max_length=512,
#         padding="max_length",
#         truncation=True,
#         return_tensors="pt"
#     )
    
#     
#     labels = tokenized["input_ids"].clone()
#     labels[labels == tokenizer.pad_token_id] = -100
#     tokenized["labels"] = labels
    
#     return tokenized

def preprocess_batch_llama(examples):
    
    inputs = tokenizer(
        examples["context"],
        examples["question"],
        truncation=True,
        padding="max_length",
        max_length=512,  
        return_tensors="pt"
    )

    
    start_positions = []
    end_positions = []

    for i, answer in enumerate(examples["answers"]):
        if len(answer['text']) == 0:  
            start_positions.append(-1)
            end_positions.append(-1)
        else:  
            if len(answer["answer_start"]) > 0:  
                start = answer["answer_start"][0]
                end = start + len(answer["text"][0]) 
                start_positions.append(start)
                end_positions.append(end)
            else:
              
                start_positions.append(-1)
                end_positions.append(-1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [9]:

# train_squad_v2 = train_squad_v2.map(
#     preprocess_batch_llama, 
#     batched=True, 
#     remove_columns=train_squad_v2.column_names
# )
# test_squad_v2 = test_squad_v2.map(
#     preprocess_batch_llama, 
#     batched=True, 
#     remove_columns=test_squad_v2.column_names
# )

testing_2 = testing_copy_2.map(
    preprocess_batch_llama, 
    batched=True, 
    remove_columns=testing_copy_2.column_names
)

# train_squad_v2 = train_squad_v2.map(preprocess_batch, batched=True, remove_columns=train_squad_v2.column_names)
# test_squad_v2 = test_squad_v2.map(preprocess_batch, batched=True, remove_columns=test_squad_v2.column_names)
# print("done")
# print(testing_2.column_names)
print(len(train_squad_v2)/len(test_squad_v2))

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

3.999961632903622


In [10]:
# !pip install rouge_score
# train_squad_v2.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])
# test_squad_v2.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])

testing_2.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])


print(train_squad_v2.column_names)

['id', 'title', 'context', 'question', 'answers']


In [11]:
print(testing_2.column_names)

['input_ids', 'attention_mask', 'start_positions', 'end_positions']


In [13]:
# !pip install rouge_score
# !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
!export TOKENIZERS_PARALLELISM=true
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
       
        input_ids = inputs.get("input_ids")
        attention_mask = inputs.get("attention_mask")
        start_positions = inputs.get("start_positions")
        end_positions = inputs.get("end_positions")

        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, 
                        start_positions=start_positions, end_positions=end_positions)

       
        loss = outputs.loss

       
        if return_outputs:
            return loss, outputs
        return loss

In [15]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# model = torch.nn.DataParallel(model, device_ids=[0, 1])
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())
print(torch.cuda.max_memory_allocated())

4943283200
4957667328
4943283200


In [17]:
import evaluate
import os
import numpy as np
from transformers import Trainer, TrainingArguments, AutoModelForQuestionAnswering, AdamW
import torch
import torch.utils.checkpoint as checkpoint

torch.cuda.empty_cache()


# model.eval()  
# torch.no_grad() 

training_args_squad_v2 = TrainingArguments(
    logging_first_step=True,  
    output_dir="./llama_squad_v2_model_test",
    eval_strategy="epoch",  
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    seed=1, 
    load_best_model_at_end=True,
    metric_for_best_model="f1", 
    save_total_limit=3,  
    overwrite_output_dir=True, 
    logging_dir="./logs", 
    logging_steps=10, 
    disable_tqdm=False,  
    report_to="tensorboard",
    fp16=True,  
    gradient_accumulation_steps=2,
    # eval_accumulation_steps =1,
    dataloader_num_workers=4,
    remove_unused_columns=False,
    no_cuda=False,
    dataloader_pin_memory=True,
)


squad_v2_metric = evaluate.load("squad_v2")
# rouge_metric = evaluate.load("rouge")
# bleu_metric = evaluate.load("bleu")
# meteor_metric = evaluate.load("meteor")


def compute_metrics_2(eval_pred):
    print("Inside compute_metrics!")
    logits, labels = eval_pred
    
   
    predictions = np.argmax(logits, axis=-1)
    print(predictions)
    print(labels)
   
    formatted_preds = [
        {"prediction_text": str(pred), "id": str(i), 'no_answer_probability': 0.} 
        for i, pred in enumerate(predictions)
    ]
    
    formatted_labels = [
        {
            "answers": {"text": [str(label)], "answer_start": [0]},
            "id": str(i)
        } 
        for i, label in enumerate(labels)
    ]
    
    print(formatted_preds)
    print(formatted_labels)
    return squad_v2_metric.compute(
        predictions=formatted_preds, 
        references=formatted_labels
    )


# def compute_metrics(eval_pred): 
#     print("Inside compute_metrics!")
#     logits, labels = eval_pred
    
#     predictions = np.argmax(logits, axis=-1)
    

#     predicted_texts = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     true_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     formatted_preds = [
#         {"prediction_text": pred, "id": str(i), 'no_answer_probability': 0.0} 
#         for i, pred in enumerate(predicted_texts)
#     ]
    
#     formatted_labels = [
#         {"answers": {"text": [true], "answer_start": [0]}, "id": str(i)} 
#         for i, true in enumerate(true_texts)
#     ]
    
#     results = squad_v2_metric.compute(predictions=formatted_preds, references=formatted_labels)
#     print(results)
#     return results 


trainer_squad_v2 = CustomTrainer(
    model=model,
    args=training_args_squad_v2,
    train_dataset=testing_2,
    eval_dataset=testing_2,
    compute_metrics=compute_metrics_2,
    tokenizer=tokenizer,
)


trainer_squad_v2.train()


trainer_squad_v2.save_model(training_args_squad_v2.output_dir)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch

Epoch,Training Loss,Validation Loss
1,6.474200,No log


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyError: "The `metric_for_best_model` training argument is set to 'eval_f1', which is not found in the evaluation metrics. The available evaluation metrics are: []. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [ ]:
from evaluate import load
squad_v2_metric = load("squad_v2")
predictions = [{'prediction_text': '1976', 'id': '56e10a3be3433e1400422b22', 'no_answer_probability': 0.0}]
references = [{'answers': {'answer_start': [97], 'text': ['1976']}, 'id': '56e10a3be3433e1400422b22'}]
results = squad_v2_metric.compute(predictions=predictions, references=references)
print(results)

#From https://huggingface.co/spaces/evaluate-metric/squad_v2